In [3]:
# ============================================
# Análisis de comentarios BNPHU con TextBlob y VADER
# Archivo: /content/sample_data/comentarios_bnphu.csv
# ============================================

# Si estás en Colab y no las tienes instaladas, ejecuta UNA VEZ descomentando:
# !pip install textblob nltk

from textblob import TextBlob
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

%matplotlib inline

# Descargar el lexicón de VADER (solo la primera vez; luego se salta si ya está)
nltk.download('vader_lexicon')

# Crear el analizador de sentimiento VADER
sia = SentimentIntensityAnalyzer()

# ============================================
# 1. Cargar el CSV de comentarios
# ============================================

ruta_csv = "https://drive.google.com/file/d/1NBioQimhCbPaTaXZSlCoqLu85boNFNai/view"
df = pd.read_csv(ruta_csv, encoding="utf-8-sig")

print("Primeras filas del dataset:")
display(df.head())

# ============================================
# 2. Funciones de clasificación de sentimiento
# ============================================

def clasificar_sentimiento(polarity, tol=0.10):
    """
    Clasificación estándar del sentimiento usando la polaridad de TextBlob.
    - Positivo: polaridad > tol
    - Negativo: polaridad < -tol
    - Neutro: en [-tol, tol]
    """
    if polarity > tol:
        return "Positivo"
    elif polarity < -tol:
        return "Negativo"
    else:
        return "Neutro"


def clasificar_institucional(sentimiento):
    """
    Clasificación adaptada a la gestión institucional:
    - Negativo -> Necesita atención
    - Positivo -> Positivo
    - Neutro   -> Neutro
    """
    if sentimiento == "Negativo":
        return "Necesita atención"
    elif sentimiento == "Positivo":
        return "Positivo"
    else:
        return "Neutro"


def clasificar_sentimiento_vader(compound, tol=0.05):
    """
    Clasificación estándar de VADER usando el 'compound' (-1 a 1).
    Umbrales recomendados:
    - Positivo: compound >= 0.05
    - Negativo: compound <= -0.05
    - Neutro: entre -0.05 y 0.05
    """
    if compound >= tol:
        return "Positivo"
    elif compound <= -tol:
        return "Negativo"
    else:
        return "Neutro"

# ============================================
# 3. Análisis con TextBlob
# ============================================

registros_tb = []

for c in df["comentario"]:
    blob = TextBlob(c)
    pol = blob.sentiment.polarity
    subj = blob.sentiment.subjectivity
    senti_std = clasificar_sentimiento(pol)
    senti_inst = clasificar_institucional(senti_std)

    registros_tb.append({
        "comentario": c,
        "polaridad_tb": pol,
        "subjetividad_tb": subj,
        "sentimiento_estandar_tb": senti_std,
        "clasificacion_institucional_tb": senti_inst
    })

df_tb = pd.DataFrame(registros_tb)

print("\n=== Resultados con TextBlob ===")
display(df_tb.head())

# Resumen TextBlob
df_sent_tb = df_tb["sentimiento_estandar_tb"].value_counts()
df_inst_tb = df_tb["clasificacion_institucional_tb"].value_counts()

print("\nResumen – Sentimiento estándar (TextBlob):\n")
print(df_sent_tb)

print("\nResumen – Clasificación institucional (TextBlob):\n")
print(df_inst_tb)

# Gráficas TextBlob
plt.figure(figsize=(6,4))
df_sent_tb.plot(kind="bar", title="Sentimiento estándar – TextBlob")
plt.xlabel("Sentimiento")
plt.ylabel("Cantidad de comentarios")
plt.xticks(rotation=0)
plt.show()

plt.figure(figsize=(6,4))
df_inst_tb.plot(kind="bar", title="Clasificación institucional – TextBlob")
plt.xlabel("Clasificación")
plt.ylabel("Cantidad de comentarios")
plt.xticks(rotation=0)
plt.show()

# Comentarios que necesitan atención según TextBlob
df_necesita_atencion_tb = df_tb[df_tb["clasificacion_institucional_tb"] == "Necesita atención"]
print("\nComentarios que 'necesitan atención' según TextBlob:")
display(df_necesita_atencion_tb)

# ============================================
# 4. Análisis con VADER (NLTK)
# ============================================

registros_vader = []

for c in df["comentario"]:
    scores = sia.polarity_scores(c)  # dict: {'neg', 'neu', 'pos', 'compound'}
    compound = scores["compound"]
    senti_std_v = clasificar_sentimiento_vader(compound)
    senti_inst_v = clasificar_institucional(senti_std_v)

    registros_vader.append({
        "comentario": c,
        "compound_vader": compound,
        "neg_vader": scores["neg"],
        "neu_vader": scores["neu"],
        "pos_vader": scores["pos"],
        "sentimiento_estandar_vader": senti_std_v,
        "clasificacion_institucional_vader": senti_inst_v
    })

df_vader = pd.DataFrame(registros_vader)

print("\n=== Resultados con VADER (NLTK) ===")
display(df_vader.head())

# Resumen VADER
df_sent_vader = df_vader["sentimiento_estandar_vader"].value_counts()
df_inst_vader = df_vader["clasificacion_institucional_vader"].value_counts()

print("\nResumen – Sentimiento estándar (VADER):\n")
print(df_sent_vader)

print("\nResumen – Clasificación institucional (VADER):\n")
print(df_inst_vader)

# Gráficas VADER
plt.figure(figsize=(6,4))
df_sent_vader.plot(kind="bar", title="Sentimiento estándar – VADER")
plt.xlabel("Sentimiento")
plt.ylabel("Cantidad de comentarios")
plt.xticks(rotation=0)
plt.show()

plt.figure(figsize=(6,4))
df_inst_vader.plot(kind="bar", title="Clasificación institucional – VADER")
plt.xlabel("Clasificación")
plt.ylabel("Cantidad de comentarios")
plt.xticks(rotation=0)
plt.show()

# Comentarios que necesitan atención según VADER
df_necesita_atencion_vader = df_vader[df_vader["clasificacion_institucional_vader"] == "Necesita atención"]
print("\nComentarios que 'necesitan atención' según VADER:")
display(df_necesita_atencion_vader)

# ============================================
# 5. Comparación TextBlob vs VADER
# ============================================

df_comparacion = pd.merge(
    df_tb[["comentario", "sentimiento_estandar_tb", "clasificacion_institucional_tb"]],
    df_vader[["comentario", "sentimiento_estandar_vader", "clasificacion_institucional_vader"]],
    on="comentario",
    how="inner"
)

print("\n=== Comparación TextBlob vs VADER ===")
display(df_comparacion)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 4462
